In [661]:
import pandas as pd
from math import log10
from random import randint

df_nirvana = pd.read_csv('Dados/nirvana_lyrics.csv')
df_pink_floyd = pd.read_csv('Dados\pink_floyd_lyrics.csv')

In [662]:
df_nirvana_treino = df_nirvana.iloc[:120]
df_pink_floyd_treino = df_pink_floyd.iloc[:120]

df_nirvana_teste = df_nirvana.iloc[120:140]
df_pink_floyd_teste = df_pink_floyd.iloc[120:140]

In [663]:
probabilidade_nirvana = len(df_nirvana_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_pink_floyd = len(df_pink_floyd_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_total = probabilidade_nirvana + probabilidade_pink_floyd

In [664]:
import re
def arruma(lyrics):
    if isinstance(lyrics, str):
        lyrics = lyrics.lower()
        lyrics = re.sub(r"[.!@#$%&*(){}:;_?/<>+|=,\[\]\n']", '', lyrics)
        lyrics = re.sub(r'[\"]', '', lyrics)
        lyrics = re.sub(r"[’]", ' ', lyrics)

    return lyrics

def split_function(texto):
    return texto.split()

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].astype(str)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].astype(str)

df_nirvana_teste.loc[:,'lyrics'] = df_nirvana_teste['lyrics'].astype(str)
df_pink_floyd_teste.loc[:,'lyrics'] = df_pink_floyd_teste['lyrics'].astype(str)


df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].apply(arruma)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].apply(arruma)

df_nirvana_teste.loc[:,'lyrics'] = df_nirvana_teste['lyrics'].apply(arruma)
df_pink_floyd_teste.loc[:,'lyrics'] = df_pink_floyd_teste['lyrics'].apply(arruma)


df_nirvana_treino = pd.DataFrame(df_nirvana_treino)
df_pink_floyd_treino = pd.DataFrame(df_pink_floyd_treino)
df_nirvana_teste = pd.DataFrame(df_nirvana_teste)
df_pink_floyd_teste = pd.DataFrame(df_pink_floyd_teste)

def keep_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return ' '.join(unique_words)

df_nirvana_treino['lyrics'] = df_nirvana_treino['lyrics'].apply(keep_unique_words)
df_pink_floyd_treino['lyrics'] = df_pink_floyd_treino['lyrics'].apply(keep_unique_words)
df_nirvana_teste['lyrics'] = df_nirvana_teste['lyrics'].apply(keep_unique_words)
df_pink_floyd_teste['lyrics'] = df_pink_floyd_teste['lyrics'].apply(keep_unique_words)

df_total_treino = pd.concat([df_nirvana_treino, df_pink_floyd_treino], ignore_index=True)
df_total_teste = pd.concat([df_nirvana_teste, df_pink_floyd_teste], ignore_index=True)


In [665]:
nirvaninha = {}
pinkinho = {}
totalzinho = {}

contador1 = 0
contador2 = 0
contador3 = 0

for musicas_n in df_nirvana_treino['lyrics']:
    nirvaninha[contador1] = musicas_n.split()
    contador1 += 1

for musicas_pf in df_pink_floyd_treino['lyrics']:
    pinkinho[contador2] = musicas_pf.split()
    contador2 += 1

for musicas_total in df_total_treino['lyrics']:
    totalzinho[contador3] = musicas_total.split()
    contador3 += 1

print(pinkinho)
print(nirvaninha)
print(totalzinho)

{0: ['stars', 'was', 'fight', 'both', 'oberon', 'blampow', 'flicker', 'discovered', 'signs', 'underground', 'knewfloating', 'not', 'green', 'powstairway', 'once', 'limpid', 'the', 'waters', 'flapflicker', 'underlime', 'saturn', 'icy', 'you', 'until', 'titanianeptune', 'down', 'moon', 'second', 'between', 'frightenblinding', 'scare', 'skies', 'in', 'scenea', 'titan', 'dan', 'dare', 'housesscorpio', 'sound', 'who', 's', 'arabian', 'blue', '1930lime', 'resoundsaround', 'miranda', 'therelime', 'surroundsthe', 'librapluto', 'undergroundjupiter', 'can', 'a', 'and'], 1: ['t', 'sam', 'found', 'catalways', 'something', 'sandhiding', 'by', 'hip', 'll', 'right', 'explain', 'on', 'side', 'sideoh', 'the', 'sifting', 'to', 'anywherethat', 'nothat', 'be', 'sidethat', 'you', 'prowling', 'left', 'siam', 'when', 'witchyou', 'night', 'sitting', 'explainjennifer', 'groundhe', 'explainat', 'your', 's', 'gentle', 'go', 'aroundthat', 're', 'he', 'seabe', 'catsomewhere', 'ship', 'explainlucifer', 'can', 'luci

In [666]:
frequencia_nirvana = {}
frequencia_pink_floyd = {}
frequencia_total = {}

for musicas in nirvaninha.values():
    for palavras in musicas:
        if palavras in frequencia_nirvana:
            frequencia_nirvana[palavras] += 1
        else:
            frequencia_nirvana[palavras] = 1

for musicas in pinkinho.values():
    for palavras in musicas:
        if palavras in frequencia_pink_floyd:
            frequencia_pink_floyd[palavras] += 1
        else:
            frequencia_pink_floyd[palavras] = 1

for musicas in totalzinho.values():
    for palavras in musicas:
        if palavras in frequencia_total:
            frequencia_total[palavras] += 1
        else:
            frequencia_total[palavras] = 1

print(frequencia_pink_floyd)
print(frequencia_nirvana)
print(frequencia_total)

{'stars': 2, 'was': 26, 'fight': 3, 'both': 6, 'oberon': 2, 'blampow': 2, 'flicker': 2, 'discovered': 2, 'signs': 2, 'underground': 2, 'knewfloating': 2, 'not': 19, 'green': 10, 'powstairway': 2, 'once': 7, 'limpid': 2, 'the': 98, 'waters': 3, 'flapflicker': 2, 'underlime': 2, 'saturn': 2, 'icy': 2, 'you': 78, 'until': 3, 'titanianeptune': 2, 'down': 27, 'moon': 5, 'second': 3, 'between': 7, 'frightenblinding': 2, 'scare': 2, 'skies': 3, 'in': 82, 'scenea': 2, 'titan': 2, 'dan': 2, 'dare': 2, 'housesscorpio': 2, 'sound': 7, 'who': 15, 's': 61, 'arabian': 2, 'blue': 6, '1930lime': 2, 'resoundsaround': 2, 'miranda': 2, 'therelime': 2, 'surroundsthe': 2, 'librapluto': 2, 'undergroundjupiter': 2, 'can': 29, 'a': 71, 'and': 69, 't': 41, 'sam': 1, 'found': 6, 'catalways': 1, 'something': 9, 'sandhiding': 1, 'by': 22, 'hip': 1, 'll': 28, 'right': 8, 'explain': 3, 'on': 52, 'side': 5, 'sideoh': 1, 'sifting': 1, 'to': 80, 'anywherethat': 1, 'nothat': 1, 'be': 37, 'sidethat': 1, 'prowling': 1, '

In [667]:
probabilidade_por_palavra_nirvana = {}
probabilidade_por_palavra_pink_floyd = {}
probabilidade_por_palavra_total = {}

for palavra,quantidade in frequencia_nirvana.items():
    probabilidade_por_palavra_nirvana[palavra] = quantidade/len(nirvaninha)

for palavra,quantidade in frequencia_pink_floyd.items():
    probabilidade_por_palavra_pink_floyd[palavra] = quantidade/len(pinkinho)

for palavra,quantidade in frequencia_total.items():
    probabilidade_por_palavra_total[palavra] = quantidade/len(totalzinho)

print(probabilidade_por_palavra_pink_floyd)
print(probabilidade_por_palavra_nirvana)
print(probabilidade_por_palavra_total)

{'stars': 0.016666666666666666, 'was': 0.21666666666666667, 'fight': 0.025, 'both': 0.05, 'oberon': 0.016666666666666666, 'blampow': 0.016666666666666666, 'flicker': 0.016666666666666666, 'discovered': 0.016666666666666666, 'signs': 0.016666666666666666, 'underground': 0.016666666666666666, 'knewfloating': 0.016666666666666666, 'not': 0.15833333333333333, 'green': 0.08333333333333333, 'powstairway': 0.016666666666666666, 'once': 0.058333333333333334, 'limpid': 0.016666666666666666, 'the': 0.8166666666666667, 'waters': 0.025, 'flapflicker': 0.016666666666666666, 'underlime': 0.016666666666666666, 'saturn': 0.016666666666666666, 'icy': 0.016666666666666666, 'you': 0.65, 'until': 0.025, 'titanianeptune': 0.016666666666666666, 'down': 0.225, 'moon': 0.041666666666666664, 'second': 0.025, 'between': 0.058333333333333334, 'frightenblinding': 0.016666666666666666, 'scare': 0.016666666666666666, 'skies': 0.025, 'in': 0.6833333333333333, 'scenea': 0.016666666666666666, 'titan': 0.01666666666666

In [668]:
ausencia_nirvana = {}
ausencia_pink_floyd = {}
ausencia_total = {}

for palavra,probabilidade in probabilidade_por_palavra_total.items():
    if palavra in probabilidade_por_palavra_pink_floyd:
        ausencia_pink_floyd[palavra] = 1 - probabilidade_por_palavra_pink_floyd[palavra]
    else:
        ausencia_pink_floyd[palavra] = 1

for palavra,probabilidade in probabilidade_por_palavra_total.items():
    ausencia_nirvana[palavra] = 1 - probabilidade
    if palavra in probabilidade_por_palavra_nirvana:
        ausencia_nirvana[palavra] = 1 - probabilidade_por_palavra_nirvana[palavra]
    else:
        ausencia_nirvana[palavra] = 1

for palavra,probabilidade in probabilidade_por_palavra_total.items():
    ausencia_total[palavra] = 1 - probabilidade
    if palavra in probabilidade_por_palavra_total:
        ausencia_total[palavra] = 1 - probabilidade_por_palavra_total[palavra]
    else:
        ausencia_total[palavra] = 1

In [669]:
def teorema_de_bayes(musica):
    musica = musica.split()
    propabilidade_final_pf = 0
    propabilidade_final_n = 0

    for palavras in musica:
        if palavras in probabilidade_por_palavra_pink_floyd:
            soma_pf = log10(probabilidade_por_palavra_pink_floyd[palavras])
            subtracao_pf = log10(probabilidade_por_palavra_total[palavras])
            propabilidade_final_pf += soma_pf
            propabilidade_final_pf -= subtracao_pf
    propabilidade_final_pf *= probabilidade_pink_floyd
        

    for palavras in musica:
        if palavras in probabilidade_por_palavra_nirvana:
            soma_n = log10(probabilidade_por_palavra_nirvana[palavras])
            subtracao_n = log10(probabilidade_por_palavra_total[palavras])
            propabilidade_final_n += soma_n
            propabilidade_final_n -= subtracao_n
    propabilidade_final_n *= probabilidade_nirvana
    
    return propabilidade_final_pf, propabilidade_final_n


In [764]:
acu_nirvana = 0
acu_pink_floyd = 0

not_acu_nirvana = 0
not_acu_pink_floyd = 0

for i in range(18):
    index = randint(120,139)
    musica1 = df_nirvana_teste['lyrics'][index]
    final_pf, final_n = teorema_de_bayes(musica1)
    if final_n > final_pf:
        acu_nirvana += 1

for i in range(18):
    index = randint(120,139)
    musica2 = df_pink_floyd_teste['lyrics'][index]
    final_pf, final_n = teorema_de_bayes(musica2)
    if final_pf > final_n:
        acu_pink_floyd += 1
        
acu_total = ((acu_nirvana + acu_pink_floyd) * 100)/36

print(f"A precisão é de {acu_total:.2f}%")

A precisão é de 86.11%
